In [212]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import os 

In [213]:
from sklearn.pipeline import make_pipeline

In [214]:
import mlflow



mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration_2")

<Experiment: artifact_location='s3://zoomcamp-mlops-2023-marlon/2', creation_time=1687327525388, experiment_id='2', last_update_time=1687327525388, lifecycle_stage='active', name='green-taxi-duration_2', tags={}>

In [215]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [216]:
df_train = read_dataframe('data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [217]:
"""
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    dv = DictVectorizer()
    model = RandomForestRegressor(**params, n_jobs=-1)

    X_train = dv.fit_transform(dict_train)
    model.fit(X_train, y_train)

    X_val = dv.transform(dict_val)
    y_pred = model.predict(X_val)

    

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

  
    mlflow.sklearn.log_model(model, artifact_path="model", )

    with open('dict_vectorizer.bin', 'wb') as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact('dict_vectorizer.bin')
"""

'\nwith mlflow.start_run():\n    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)\n    mlflow.log_params(params)\n\n    dv = DictVectorizer()\n    model = RandomForestRegressor(**params, n_jobs=-1)\n\n    X_train = dv.fit_transform(dict_train)\n    model.fit(X_train, y_train)\n\n    X_val = dv.transform(dict_val)\n    y_pred = model.predict(X_val)\n\n    \n\n    rmse = mean_squared_error(y_pred, y_val, squared=False)\n    print(params, rmse)\n    mlflow.log_metric(\'rmse\', rmse)\n\n  \n    mlflow.sklearn.log_model(model, artifact_path="model", )\n\n    with open(\'dict_vectorizer.bin\', \'wb\') as f_out:\n        pickle.dump(dv, f_out)\n\n    mlflow.log_artifact(\'dict_vectorizer.bin\')\n'

In [218]:


with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

  
    mlflow.sklearn.log_model(pipeline, artifact_path="model")
    

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


In [104]:
from mlflow.tracking import MlflowClient


In [82]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '4cee97f1bfd44f7c8dbc49acc00c1264'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [83]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

/tmp/ipykernel_138100/2836594815.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')


In [84]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [85]:
dv

DictVectorizer()